In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

inputs = tf.keras.layers.Input(shape=(5,))
hidden = tf.keras.layers.Dense(2000, activation="relu", name="hidden", trainable = False)
outputs = tf.keras.layers.Dense(1)(hidden(inputs))
model = tf.keras.models.Model(inputs = inputs, outputs = outputs)


In [20]:
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
hidden (Dense)               (None, 2000)              12000     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2001      
Total params: 14,001
Trainable params: 2,001
Non-trainable params: 12,000
_________________________________________________________________


In [21]:
model.compile(optimizer='sgd', loss = 'mse')
model.metrics_names

[]

In [22]:
import numpy as np
x1, x2 = np.random.normal(size = (200, 5)), np.random.normal(size = (20, 5))
beta, delta = np.array([1, 1, 0, 0, 0]).reshape((-1,1)), np.array([0, 0, 0.2, 0.2, 0.2]).reshape((-1,1))
y1, y2 = x1 @ (beta + delta) + 0.1 * np.random.normal(size=(200, 1)), x2 @ (beta - delta) + 0.1 * np.random.normal(size = (20, 1))
x, y = np.vstack((x1, x2)), np.vstack((y1, y2))

In [23]:
x_test = np.random.normal(size = (1000, 5))
y_test = x_test @ (beta + delta) + 0.1 * np.random.normal(size = (1000, 1))

model.fit(x, y, epochs = 200)

model.evaluate(x_test, y_test)

In [24]:
from scipy.linalg import inv
w = np.random.normal(size = (5, 2000))

In [25]:
def hidden_layer(x, w):
    z = x @ w
    z[z<0]=0
    return z

In [26]:
z = hidden_layer(x, w)
beta = z.T @ inv(z @ z.T) @ y

In [28]:
def predict(x, weights):
    w, beta = weights
    z = hidden_layer(x, w)
    return z @ beta

def evaluate(data, weights, sample_weight = []):
    x, y = data
    n, _ = x.shape
    if sample_weight == []:
        sample_weight = np.array([1]*n)
    sample_weight = sample_weight/np.sum(sample_weight)
    y_hat = predict(x, weights)
    error = (y - y_hat).reshape((-1,))
    return np.sum(error ** 2  * sample_weight)

In [30]:
y_hat = predict(x, [w, beta])

In [33]:
y_hat- y

array([[ 1.47371004e-12],
       [ 5.82589532e-13],
       [ 2.36499709e-12],
       [ 3.92308408e-12],
       [ 5.56354962e-12],
       [ 4.53304061e-13],
       [ 6.67549349e-13],
       [ 6.45594689e-12],
       [ 3.46789264e-12],
       [ 4.84123852e-12],
       [ 1.82331927e-12],
       [ 2.72382117e-12],
       [ 4.36095604e-13],
       [ 3.90998345e-12],
       [ 2.76934031e-12],
       [ 3.54294372e-12],
       [ 1.50696122e-12],
       [ 5.03685982e-12],
       [ 1.56485935e-12],
       [ 2.39297471e-12],
       [ 1.96465066e-12],
       [ 6.42019771e-12],
       [ 8.46811510e-12],
       [ 6.32272013e-12],
       [ 3.16902060e-12],
       [ 1.53521640e-12],
       [ 2.85582669e-12],
       [ 2.97822877e-12],
       [ 4.66915395e-12],
       [ 9.06374975e-12],
       [ 2.20423679e-12],
       [ 1.94511074e-12],
       [-4.15445456e-13],
       [ 1.38244971e-12],
       [ 3.35409478e-12],
       [ 8.90287843e-13],
       [ 1.97841743e-13],
       [ 8.95228336e-13],
       [ 5.7